In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt 
from ECD_control.ECD_optimization.tf_adam_optimizer import AdamOptimizer
from ECD_control.gate_sets.SNAP_disp import SNAP_disp
from ECD_control.ECD_optimization.GateSynthesizer import GateSynthesizer
from ECD_control.ECD_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


Need tf version 2.3.0 or later. Using tensorflow version: 2.8.0



c:\users\jacob\ecd_control\ECD_control\ECD_optimization\GateSynthesizer.py:268: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if parameter is "initial_states" or "final_states":



Need tf version 2.3.0 or later. Using tensorflow version: 2.8.0



[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11198794031164360851
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7806648320
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11894110682621132772
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [2]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 40
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'N_snap' : 10,
    'beta_scale' : 3.0
}
SNAP_gate_set = SNAP_disp(**gate_set_params)

In [3]:
#The target oscillator state.
fock = 4
psi_t = qt.basis(N,fock) #target state

In [4]:
#the optimization options
synth_params = {
    'gateset' : SNAP_gate_set,
    'N_blocks':6,
    'N_multistart' : 1000, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 3.0, #maximum |beta| for random initialization
    'initial_states' : [qt.basis(N,0)], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [psi_t], #end in |e> |target>.
    'name' : 'SNAP Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [5]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

filename: SNAP Fock 4.h5

Best circuit parameters found:
betastf.Tensor(
[-0.49573-2.32275j -0.1262 +0.64122j  2.74676+0.1775j  -2.18921-0.72849j
  0.0519 +0.70454j -0.06927-0.78955j], shape=(6,), dtype=complex64)
thetastf.Tensor(
[[ 2.93468  1.89845  1.50543  2.73436  2.72077 -1.63621 -1.71612 -0.45325
   3.03798 -2.40076]
 [-0.05577 -1.44144  1.93759  0.01158  1.54219 -2.84614 -0.40083 -2.91761
   2.57775 -0.89728]
 [ 0.06393  2.11106  1.88218  1.32907 -2.2634  -1.52431 -0.57317 -2.02928
  -2.86327  0.06485]
 [ 0.27285  0.0012   1.00303  1.88743 -2.20055  1.29821 -1.14954 -0.86878
  -2.13344  2.65105]
 [-0.18015  2.21831  1.68049 -0.02724  0.20559 -0.00563  1.06114 -1.40491
   0.71707  0.23668]
 [ 0.39328  1.46564 -2.57492  1.17359 -0.59531  0.97441 -1.73528 -1.02749
  -2.18867 -1.3912 ]], shape=(6, 10), dtype=float32)




In [6]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

Start time: 2022-02-17 16:03:59
 Epoch: 5 / 100 Max Fid: 0.997083 Avg Fid: 0.658514 Max dFid: 0.252935 Avg dFid: 0.086105 Elapsed time: 0:00:01.935998 Remaing time: 0:00:36.783966

 Optimization stopped. Term fidelity reached.

filename: SNAP Fock 4.h5

Best circuit parameters found:
betastf.Tensor(
[ 0.43181-0.12439j  1.40035-0.04461j -0.25261+0.41103j -0.2698 +0.17838j
  0.00925+0.00958j -0.40497-0.15609j], shape=(6,), dtype=complex64)
thetastf.Tensor(
[[-0.39269 -2.94413 -1.74506 -0.10756  1.73452  3.14069 -2.67062 -0.46888
  -0.88737 -1.11557]
 [-0.53557 -0.45392  2.34284 -0.22747 -2.94304  1.6987   1.48831  0.04544
   1.56422  1.9661 ]
 [-2.45539  1.26105 -1.69255  1.39697 -1.02213  0.25015 -1.12272  1.91541
   0.63636  2.59777]
 [-2.15979  2.91085  2.67096  0.93215  2.41757  1.34316 -3.11571 -0.49621
   0.02258 -0.8836 ]
 [-2.87618  1.2336   1.60462  2.63829  0.41146  0.91004 -1.55743 -2.0037
  -0.9197   2.23323]
 [ 2.53773  2.26331  1.43097  2.78892 -0.47886  3.00681  2.01226 -0

'2022-02-17 16:03:59'

In [7]:
#can print info, including the best circuit found.
gatesynth.print_info() 

filename: SNAP Fock 4.h5

Best circuit parameters found:
betastf.Tensor(
[ 0.43181-0.12439j  1.40035-0.04461j -0.25261+0.41103j -0.2698 +0.17838j
  0.00925+0.00958j -0.40497-0.15609j], shape=(6,), dtype=complex64)
thetastf.Tensor(
[[-0.39269 -2.94413 -1.74506 -0.10756  1.73452  3.14069 -2.67062 -0.46888
  -0.88737 -1.11557]
 [-0.53557 -0.45392  2.34284 -0.22747 -2.94304  1.6987   1.48831  0.04544
   1.56422  1.9661 ]
 [-2.45539  1.26105 -1.69255  1.39697 -1.02213  0.25015 -1.12272  1.91541
   0.63636  2.59777]
 [-2.15979  2.91085  2.67096  0.93215  2.41757  1.34316 -3.11571 -0.49621
   0.02258 -0.8836 ]
 [-2.87618  1.2336   1.60462  2.63829  0.41146  0.91004 -1.55743 -2.0037
  -0.9197   2.23323]
 [ 2.53773  2.26331  1.43097  2.78892 -0.47886  3.00681  2.01226 -0.56404
   1.9979  -2.49609]], shape=(6, 10), dtype=float32)




In [8]:
print(gatesynth.best_fidelity())

0.9971835


In [9]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
thetas = best_circuit['thetas']
print(best_circuit)

{'betas': <tf.Tensor: shape=(6,), dtype=complex64, numpy=
array([ 0.43180716-0.12439014j,  1.4003489 -0.04460944j,
       -0.25261053+0.4110305j , -0.2698    +0.17837797j,
        0.00925414+0.00958203j, -0.40497187-0.15608501j], dtype=complex64)>, 'thetas': <tf.Tensor: shape=(6, 10), dtype=float32, numpy=
array([[-0.39269114, -2.9441268 , -1.7450646 , -0.1075635 ,  1.7345192 ,
         3.140692  , -2.6706219 , -0.46888328, -0.8873651 , -1.1155713 ],
       [-0.5355723 , -0.45391583,  2.342839  , -0.2274692 , -2.9430397 ,
         1.6987035 ,  1.4883053 ,  0.04543924,  1.5642169 ,  1.9661019 ],
       [-2.45539   ,  1.2610524 , -1.6925521 ,  1.3969672 , -1.0221283 ,
         0.2501483 , -1.1227245 ,  1.9154088 ,  0.63635635,  2.597768  ],
       [-2.159785  ,  2.9108508 ,  2.6709597 ,  0.9321463 ,  2.4175675 ,
         1.3431599 , -3.1157053 , -0.49621153,  0.02257776, -0.88359594],
       [-2.876178  ,  1.233598  ,  1.6046245 ,  2.6382868 ,  0.41146016,
         0.9100363 , -1.557434 